<a href="https://colab.research.google.com/github/sushirito/Molecular-Dynamics/blob/main/tleap_simplification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install AmberTools
!pip install -q condacolab
import condacolab
condacolab.install()  # Installs Conda
!mamba install -c conda-forge ambertools -y

✨🍰✨ Everything looks OK!

Looking for: ['ambertools']

conda-forge/linux-64                                        Using cache
conda-forge/noarch                                          Using cache

Pinned packages:
  - python 3.11.*
  - python 3.11.*
  - python_abi 3.11.* *cp311*
  - cuda-version 12.*


Transaction

  Prefix: /usr/local

  All requested packages already installed



In [2]:
!pip install openmm
!pip install MDAnalysis

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 94.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 93.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 86.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 88.8 MB/s eta 0:00:00


Final

In [21]:
import os
from openmm.app import *
from openmm import *
from openmm.unit import nanometer, kelvin, picosecond, picoseconds, kilojoule, mole, amu, kilojoules_per_mole
import numpy as np
from openmm.unit import kilojoules_per_mole

# Modified TLEAP script with proper ion placement
tleap_script = """
source leaprc.protein.ff14SB
source leaprc.water.tip3p
loadAmberParams frcmod.ions234lm_126_tip3p

# Create empty system with sufficient box size
solv = createUnit solv
solvateBox solv TIP3PBOX 20.0 iso  # Isotropic box
addIons solv HG 1                  # Use addIons instead of addIons2
addIons solv Cl- 2

saveAmberParm solv solv.prmtop solv.inpcrd
quit
"""

# Run TLEAP and load parameters
with open('tleap_fixed.in', 'w') as f:
    f.write(tleap_script)

os.system("tleap -f tleap_fixed.in > tleap.log 2>&1")

# Load AMBER files
prmtop = AmberPrmtopFile('solv.prmtop')
inpcrd = AmberInpcrdFile('solv.inpcrd')

# System setup with proper constraints
system = prmtop.createSystem(
    nonbondedMethod=PME,
    nonbondedCutoff=1.0*nanometer,
    constraints=HBonds,
    hydrogenMass=1.5*amu,  # Mass repartitioning
    rigidWater=True
)

# Initialize simulation correctly
integrator = LangevinMiddleIntegrator(
    300*kelvin,
    1/picosecond,
    0.001*picoseconds
)

# Explicitly create Simulation instance
simulation = Simulation(
    prmtop.topology,
    system,
    integrator,
    Platform.getPlatformByName('CPU')
)

# Set positions and validate
if inpcrd.boxVectors is not None:
    simulation.context.setPeriodicBoxVectors(*inpcrd.boxVectors)
simulation.context.setPositions(inpcrd.positions)

# Energy validation before minimization
state = simulation.context.getState(getEnergy=True)
if np.isnan(state.getPotentialEnergy()._value):
    raise ValueError("Initial energy contains NaN - check system parameters")

# Modified minimization protocol with proper units
simulation.minimizeEnergy(
    tolerance=50*kilojoules_per_mole,  # Use predefined unit
    maxIterations=1000
)
simulation.minimizeEnergy(
    tolerance=0.1*kilojoules_per_mole,  # Use predefined unit
    maxIterations=5000
)

# Configure reporters
simulation.reporters.append(PDBReporter('traj.pdb', 500))
simulation.reporters.append(StateDataReporter(stdout, 500, step=True, temperature=True))

# Run production MD
print("Starting production simulation...")
simulation.step(5000)
print("Simulation completed successfully")


TypeError: Unit "kilojoule/mole" is not compatible with Unit "kilojoule/(nanometer*mole)".

#REAL WORKING

In [23]:
tleap_script = """

source leaprc.protein.ff14SB

source leaprc.water.tip3p

loadAmberParams frcmod.ions234lm_126_tip3p

solv = createUnit solv

solvateBox solv TIP3PBOX 10.0

addIons2 solv HG 1

addIons2 solv Cl- 2

saveAmberParm solv solv.prmtop solv.inpcrd

quit

"""

with open('tleap.in', 'w') as f:
    f.write(tleap_script)

!tleap -f tleap.in

-I: Adding /usr/local/dat/leap/prep to search path.
-I: Adding /usr/local/dat/leap/lib to search path.
-I: Adding /usr/local/dat/leap/parm to search path.
-I: Adding /usr/local/dat/leap/cmd to search path.
-f: Source tleap.in.

Welcome to LEaP!
(no leaprc in search path)
Sourcing: ./tleap.in
----- Source: /usr/local/dat/leap/cmd/leaprc.protein.ff14SB
----- Source of /usr/local/dat/leap/cmd/leaprc.protein.ff14SB done
Log file: ./leap.log
Loading parameters: /usr/local/dat/leap/parm/parm10.dat
Reading title:
PARM99 + frcmod.ff99SB + frcmod.parmbsc0 + OL3 for RNA
Loading parameters: /usr/local/dat/leap/parm/frcmod.ff14SB
Reading force field modification type file (frcmod)
Reading title:
ff14SB protein backbone and sidechain parameters
Loading library: /usr/local/dat/leap/lib/amino12.lib
Loading library: /usr/local/dat/leap/lib/aminoct12.lib
Loading library: /usr/local/dat/leap/lib/aminont12.lib
----- Source: /usr/local/dat/leap/cmd/leaprc.water.tip3p
----- Source of /usr/local/dat/leap/cm

In [36]:
tleap_script = """

source leaprc.protein.ff14SB

source leaprc.water.tip3p

loadAmberParams frcmod.ions234lm_126_tip3p
list
solv = createUnit solv

solvateBox solv TIP3PBOX 20.0

addIons2 solv HG 1

addIons2 solv Cl- 2

saveAmberParm solv solv.prmtop solv.inpcrd

quit

"""

with open('tleap.in', 'w') as f:
    f.write(tleap_script)

!tleap -f tleap.in

# OpenMM simulation code
from simtk.openmm import *
from simtk.openmm.app import *
from simtk.unit import *
import sys


# Load AMBER files
prmtop = AmberPrmtopFile('solv.prmtop')
inpcrd = AmberInpcrdFile('solv.inpcrd')

# Create system with constraints and PME
system = prmtop.createSystem(
    nonbondedMethod=PME,
    nonbondedCutoff=1.0*nanometers,
    constraints=HBonds
)

# Set up simulation
integrator = LangevinIntegrator(300*kelvin, 1/picosecond, 0.002*picoseconds)
simulation = Simulation(prmtop.topology, system, integrator)
simulation.context.setPositions(inpcrd.positions)

# Energy minimization
simulation.minimizeEnergy()

# Configure reporters
simulation.reporters.append(DCDReporter('trajectory.dcd', 100))  # Trajectory file
simulation.reporters.append(StateDataReporter(sys.stdout, 1000, step=True,
    potentialEnergy=True, temperature=True))


# Run production MD
simulation.step(5000)  # 10 ps simulation (0.002 ps/step)


-I: Adding /usr/local/dat/leap/prep to search path.
-I: Adding /usr/local/dat/leap/lib to search path.
-I: Adding /usr/local/dat/leap/parm to search path.
-I: Adding /usr/local/dat/leap/cmd to search path.
-f: Source tleap.in.

Welcome to LEaP!
(no leaprc in search path)
Sourcing: ./tleap.in
----- Source: /usr/local/dat/leap/cmd/leaprc.protein.ff14SB
----- Source of /usr/local/dat/leap/cmd/leaprc.protein.ff14SB done
Log file: ./leap.log
Loading parameters: /usr/local/dat/leap/parm/parm10.dat
Reading title:
PARM99 + frcmod.ff99SB + frcmod.parmbsc0 + OL3 for RNA
Loading parameters: /usr/local/dat/leap/parm/frcmod.ff14SB
Reading force field modification type file (frcmod)
Reading title:
ff14SB protein backbone and sidechain parameters
Loading library: /usr/local/dat/leap/lib/amino12.lib
Loading library: /usr/local/dat/leap/lib/aminoct12.lib
Loading library: /usr/local/dat/leap/lib/aminont12.lib
----- Source: /usr/local/dat/leap/cmd/leaprc.water.tip3p
----- Source of /usr/local/dat/leap/cm

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-36-d5b190801fdc>", line 61, in <cell line: 0>
    simulation.step(5000)  # 10 ps simulation (0.002 ps/step)
    ^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/site-packages/openmm/app/simulation.py", line 150, in step
    self._simulate(endStep=self.currentStep+steps)
  File "/usr/local/lib/python3.11/site-packages/openmm/app/simulation.py", line 229, in _simulate
    self.integrator.step(10) # Only take 10 steps at a time, to give Python more chances to respond to a control-c.
    ^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/site-packages/openmm/openmm.py", line 3248, in step
    return _openmm.LangevinMiddleIntegrator_step(self, steps)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
KeyboardInterrupt

During handling of the above exc

TypeError: object of type 'NoneType' has no len()

In [39]:
tleap_script = """

source leaprc.protein.ff14SB

source leaprc.water.tip3p

loadAmberParams frcmod.ions234lm_126_tip3p
list
solv = createUnit solv

solvateBox solv TIP3PBOX 20.0

addIons2 solv HG 1

addIons2 solv Cl- 2

saveAmberParm solv solv.prmtop solv.inpcrd

quit

"""

with open('tleap.in', 'w') as f:
    f.write(tleap_script)

!tleap -f tleap.in

from simtk.openmm import *
from simtk.openmm.app import *
from simtk.unit import *
import sys

# Load AMBER files
prmtop = AmberPrmtopFile('solv.prmtop')
inpcrd = AmberInpcrdFile('solv.inpcrd')

# Create system with constraints and PME
system = prmtop.createSystem(
    nonbondedMethod=PME,
    nonbondedCutoff=1.0*nanometers,
    constraints=HBonds
)

# Set up simulation
integrator = LangevinIntegrator(300*kelvin, 1/picosecond, 0.002*picoseconds)
simulation = Simulation(prmtop.topology, system, integrator)
simulation.context.setPositions(inpcrd.positions)

# Energy minimization
simulation.minimizeEnergy()

# **Save the initial structure as a PDB for visualization in VMD**
with open('solv_init.pdb', 'w') as f:
    PDBFile.writeFile(prmtop.topology, simulation.context.getState(getPositions=True).getPositions(), f)

# Configure reporters
# Configure PDB trajectory reporting
simulation.reporters.append(
    PDBReporter('trajectory.pdb', 100)  # Save every 1000 steps (4 ps)
)

# Add energy reporting
simulation.reporters.append(
    StateDataReporter(sys.stdout, 100,
                     step=True, potentialEnergy=True, temperature=True)
)

# Run production MD
simulation.step(5000)  # 1 ns simulation (4 fs/step)


# Save the final structure
with open('solv_final.pdb', 'w') as f:
    PDBFile.writeFile(prmtop.topology, simulation.context.getState(getPositions=True).getPositions(), f)


-I: Adding /usr/local/dat/leap/prep to search path.
-I: Adding /usr/local/dat/leap/lib to search path.
-I: Adding /usr/local/dat/leap/parm to search path.
-I: Adding /usr/local/dat/leap/cmd to search path.
-f: Source tleap.in.

Welcome to LEaP!
(no leaprc in search path)
Sourcing: ./tleap.in
----- Source: /usr/local/dat/leap/cmd/leaprc.protein.ff14SB
----- Source of /usr/local/dat/leap/cmd/leaprc.protein.ff14SB done
Log file: ./leap.log
Loading parameters: /usr/local/dat/leap/parm/parm10.dat
Reading title:
PARM99 + frcmod.ff99SB + frcmod.parmbsc0 + OL3 for RNA
Loading parameters: /usr/local/dat/leap/parm/frcmod.ff14SB
Reading force field modification type file (frcmod)
Reading title:
ff14SB protein backbone and sidechain parameters
Loading library: /usr/local/dat/leap/lib/amino12.lib
Loading library: /usr/local/dat/leap/lib/aminoct12.lib
Loading library: /usr/local/dat/leap/lib/aminont12.lib
----- Source: /usr/local/dat/leap/cmd/leaprc.water.tip3p
----- Source of /usr/local/dat/leap/cm

In [22]:
from openmm import app, unit
from openmm.app import PDBFile, Simulation
from openmm.vec3 import Vec3
import parmed as pmd

# Load AMBER files
parm = pmd.load_file('solv.prmtop', 'solv.inpcrd')
system = parm.createSystem(nonbondedMethod=app.PME,
                          nonbondedCutoff=1.0*unit.nanometer,
                          constraints=app.HBonds)

# Set up simulation
integrator = app.LangevinMiddleIntegrator(
    300*unit.kelvin,
    1/unit.picosecond,
    0.002*unit.picoseconds
)

platform = app.Platform.getPlatformByName('CPU')  # or 'CPU', 'OpenCL'
properties = {'CudaPrecision': 'mixed'}

simulation = Simulation(parm.topology, system, integrator, platform, properties)
simulation.context.setPositions(parm.positions)
simulation.context.setPeriodicBoxVectors(*parm.box_vectors)

# Energy minimization
simulation.minimizeEnergy()

# Equilibration
simulation.reporters.append(app.StateDataReporter(stdout, 1000, step=True,
    potentialEnergy=True, temperature=True))
simulation.step(5000)  # 10 ps equilibration

# Production
simulation.reporters.append(app.DCDReporter('trajectory.dcd', 1000))
simulation.reporters.append(app.StateDataReporter('stats.csv', 1000,
    step=True, time=True, potentialEnergy=True, temperature=True,
    density=True))
simulation.step(250000)  # 500 ps production


AttributeError: module 'openmm.app' has no attribute 'LangevinMiddleIntegrator'